In [1]:
import pandas as pd

In [2]:
import time                                                 # para tiempo
inicio=time.time()                                          # momento inicial
print ('Comenzando...\n')
print ('Modelo SoftMax (MLR) : Numeros')    
import pandas as pd                                         # dataframe
import numpy as np                                          # numerical python, algebra lineal

import matplotlib.pyplot as plt                             # plots, graficos
import seaborn as sns                                       # plots
%matplotlib inline

from sklearn.metrics import confusion_matrix                # metricas, matriz de confusion
from scipy.optimize import minimize                         # minimizar, opt

from sklearn.datasets import make_circles, load_boston      # datasets
from sklearn.model_selection import train_test_split as tts

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import warnings                                             # avisos
warnings.filterwarnings('ignore', category=RuntimeWarning)  # elimino un warning por valores NaN en logaritmos o /0

Comenzando...

Modelo SoftMax (MLR) : Numeros


In [3]:
def f(X,a):                                 # funcion logistica, sigmoide, funcion del modelo, con z=X*alfa, el producto escalar
    return 1.0/(1.0+np.exp(-np.dot(X,a)))   # Boltzmann con pivote, alfa[i]=0

In [4]:
def coste(X,a,y,lambda_reg):              # funcion coste, funcion a minimizar 
    return -(np.sum(np.log(f(X,a)))+np.dot((y-1).T,(np.dot(X,a))))/y.size + lambda_reg/(2.0*y.size)*np.dot(a[1:],a[1:])

In [5]:
def grad_coste(X,a,y,lambda_reg):          # gradiente de la funcion coste con regularizacion
    return (np.dot(X.T,(f(X,a)-y)))/y.size + lambda_reg/(2.0*y.size)*np.concatenate(([0], a[1:])).T

In [6]:
def normalizador(X):                # normalizador de X
    X_media=X.mean(axis=0)          # media de X
    X_std=X.std(axis=0)             # desviacion estandar de X
    X_std[X_std==0]=1.0             # si hay alguna std=0 ponla a 1
    X=(X-X_media)/X_std             # normaliza
    
    X=np.insert(X, 0, 1, axis=1)    # esta linea añade una columna de 1, feature engineering [1, f1, f2.., fn, f1f2...] (mejora un 10%)
    
    return X

In [ ]:
datos = pd.read_csv('DF.csv')
print ('Datos leidos...')
display(datos.head())
matriz_datos=datos.values   
print ('Dimensiones matriz de datos: {}'.format(matriz_datos.shape))

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(matriz_datos[45,1:].reshape(20,20))  
plt.show()

In [ ]:
Y=np.zeros((matriz_datos.shape[0],10))   
print ('Dimension de Y: {}'.format(Y.shape))  
print ('')
for i in range(10):
    Y[:,i]=np.where(matriz_datos[:,0]==i,1,0)
print(Y[0:10,:]) # 10 primeras filas

In [ ]:
etiquetas=matriz_datos[:,0]        # etiqueta, el numero en si, 42000 etiquetas
X=matriz_datos[:,1:]               # datos numericos de los pixeles, cada columna es un pixel (variables indep)
print ('Dimension original de X: {}'.format(X.shape)) 
print ('')
X=X[:,X.sum(axis=0)!=0]            # se quitan las columnas=0 (la suma de los elementos es no nulo, no hay informacion)
print ('Dimension limpia de X: {}'.format(X.shape))

In [ ]:
X_train, Y_train=X[0:30000,:], Y[0:30000,:]        # datos de entranamiento
X_train_sk=X_train.copy()                          # para sklearn

X_test, Y_test=X[30000:,:], Y[30000:,:]            # datos de test
X_test_sk=X_test.copy()                            # para sklearn

print ('Dimensiones train: X={}, Y={}'.format(X_train.shape, Y_train.shape))  
print ('')
print ('Dimensiones test: X={}, Y={}'.format(X_test.shape, Y_test.shape))

In [ ]:
etiquetas_train=etiquetas[0:30000]       # etiquetas para entranamiento
etiquetas_test=etiquetas[30000:]         # etiquetas para test

print ('Dimensiones etiquetas train: {}'.format(etiquetas_train.shape))
print ('')
print ('Dimensiones etiquetas test: {}'.format(etiquetas_test.shape))

In [ ]:
X_train=normalizador(X_train)
X_test=normalizador(X_test)
print ('Datos normalizados.')

In [ ]:
val_inicial=np.random.rand(X_train.shape[1]) # valores iniciales de los parametros alfa

A_opt=np.zeros((X_train.shape[1],10))        # se crea la matriz para los parametros optimizados, alfas

In [ ]:
lambda_reg=100.            # valor obtenido desde gridsearching

In [ ]:
inicio_opt=time.time()                       # inicio optimizacion
for i in range(10):
    print ('\n\nOptimizando {} frente al resto.'.format(i))

    def opt_coste(a):                        # funcion a minimizar
        return coste(X_train, a, Y_train[:,i], lambda_reg) 

    def opt_grad_coste(a):                   # gradiente 
        return grad_coste(X_train, a, Y_train[:,i], lambda_reg)	

    # metodo Nelder-Mead, Powell, CG, BFGS, Newton-CG, L-BFGS-B, TNC, COBYLA, SLSQP, trust-constr, 
    # dogleg, trust-ncg, trust-exact, trust-krylov (tambien custom)            
    metodo='trust-constr'
    print ('Optimizacion {}...'.format(metodo)) # minimizacion, optimizacion
    i_opt=time.time() 
    modelo=minimize(opt_coste, val_inicial, method=metodo, jac=opt_grad_coste, tol=1e-4, options={'disp':True}) 
    print ('Hecho.')
    print ("Tiempo optimizacion: {:.2f} segundos." .format(time.time()-i_opt))  
    A_opt[:,i]=modelo.x

t_custom=time.time()-inicio_opt   # tiempo desde inicio hasta final minimizacion
print ('\nTiempo total optimizacion custom: {:.2f} segundos.\n' .format(t_custom))

In [ ]:
y_pred=[]                  # etiquetas predichas
y_prob=[]                  # probabilidades de las etiquetas predichas

def resumen(datos):        # testeo
    for e in datos:
        nombre, etiqueta, Xs=e         
        etiq=etiqueta.size
        probs=np.zeros((etiq,2))      # etiquetas con su probabilidad
        cuenta=0                      # conteo de aciertos
        for muestra in range(etiq): 
            for n in range(10):
                alfa=A_opt[:,n]       # parametros de softmax
                probs[n,0]=n
                probs[n,1]=f(Xs[muestra,:],alfa)      # evaluacion de la prediccion
                             
            probs=probs[probs[:,1].argsort()[::-1]]   # se pone la prob mas alta al principio
            y_pred.append(probs[0,0])
            y_prob.append(probs[0,1])
            if probs[0,0]==etiqueta[muestra]:         # si se acierta +1
                cuenta+=1
        print ("\n{}".format(nombre))
        print ("{} correctos de {} ==> {:.4}% correcto".format(cuenta, etiq, cuenta/etiq*100))

In [ ]:
resumen([('Entranamiento  :', etiquetas_train, X_train)])
resumen([('Test  :', etiquetas_test, X_test)])

In [ ]:
print ('Dimensiones matriz de parametros={}'.format(A_opt.shape))
df=pd.DataFrame(A_opt, columns=[i+1 for i in range(A_opt.shape[1])])  # se guardan los parametros softmax en csv
#df.to_csv('alfas.csv', index=False)

## sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
ini_opt_sk=time.time()
logreg=LogisticRegression(C=0.01, penalty='l2', tol=0.0001, max_iter=70,
                          solver='lbfgs', multi_class='multinomial').fit(X_train_sk, etiquetas_train)
t_sklearn=time.time()-ini_opt_sk
print ('\nTiempo total optimizacion sklearn: {:.2f} segundos.\n' .format(t_sklearn))

In [ ]:
def resumen_sk(datos):
    for e in datos:
        nombre, etiqueta, Xs=e
        etiq=etiqueta.size
        
        y_pred_sk=logreg.predict(Xs)
        
        cuenta=0
        for muestra in range(etiq):
            if y_pred_sk[muestra]==etiqueta[muestra]:         
                cuenta+=1
        
        print ("\n{}".format(nombre))
        print ("{} correctos de {} ==> {:.4}% correcto".format(cuenta, etiq, cuenta/etiq*100))

In [ ]:
resumen_sk([('Entranamiento  :', etiquetas_train, X_train_sk)])
resumen_sk([('Test  :', etiquetas_test, X_test_sk)])

In [ ]:
print ('SkLearn es {:.2f} veces más rapido.'.format(t_custom/t_sklearn))

In [ ]:
y_pred_sk=logreg.predict(X_test_sk)
comp=[y_pred[30000:][i]==y_pred_sk[i] for i in range(len(y_pred[30000:]))]
n_equal=len([e for e in comp if e==False])/len(y_pred_sk)

print ('Hay una diferencia entre ambos modelos del {:.2f}%.'.format(n_equal*100))

In [ ]:
resumen([('Entranamiento Custom:', etiquetas_train, X_train)])
resumen([('Test Custom:', etiquetas_test, X_test)])
print ('')
resumen_sk([('Entranamiento SkLearn:', etiquetas_train, X_train_sk)])
resumen_sk([('Test SkLearn:', etiquetas_test, X_test_sk)])